Given facial data, the goal of this notebook is to write a program to fit a logistic regression model to the training data to recognize human faces.

In [1]:
from scipy.io import loadmat
import numpy as np
import pandas as pd

## Import data for facial detection
The file `faces.mat` is composed of `train_faces`, `train_nonfaces`, `test_faces`, and `test_nonface`. Training dataset is constructed to be a 361 by 4858 matrix, and test data is constructed to be a 361 by 944 matrix.

In [2]:
faces = loadmat('faces.mat')

train_faces, train_nonfaces = faces['train_faces'], faces['train_nonfaces']
test_faces, test_nonfaces = faces['test_faces'], faces['test_nonfaces']

# Training data, with face as 1 and non-face as 0
X_train = np.vstack([train_faces, train_nonfaces])
y_train = np.vstack([np.ones(train_faces.shape[0]).reshape(-1,1),
                     np.zeros(train_faces.shape[0]).reshape(-1,1)])

# Testing data, with face as 1 and non-face as 0
X_test = np.vstack([test_faces, test_nonfaces])
y_test = np.vstack([np.ones(test_faces.shape[0]).reshape(-1,1),
                     np.zeros(test_faces.shape[0]).reshape(-1,1)])

## Functions for Logistic Regression

- Sigmoid function: used to compute pi's in subsequent steps

In [3]:
def sigmoid(x):
    return (np.exp(x) / (1 + np.exp(x)))

- Vector p maps training samples with their corresponding $p_i$

In [4]:
def p_vector(X,beta):
    p = np.zeros(X.shape[0]).reshape(-1,1)
    for i in np.arange(0,X.shape[0]):
        X_i = X[i,:].reshape(-1,1)
        p[i] = sigmoid(beta.T @ X_i)
    return p.reshape(-1,1)

- Euclidean norm is used to test convergence, has the formula $||x-y||_2$

In [5]:
def norm(x,y):
    return np.sqrt(np.sum(np.power(x-y,2)))

- Newton-Raphson method is to find estimate $\beta$

In [6]:
# 
def get_beta(X,y, max_iter = 300, tol = 1e-10):
    converged, count = False, 0
    beta = np.zeros(X_train.shape[1]).reshape(-1,1)
    
    while not converged:
        p = p_vector(X,beta)
        W = np.diag(np.multiply(p,1-p).reshape(-1,1).T[0])
        # define - H^{-1} S
        inc = np.linalg.inv(X.T @ W @ X) @ (X.T @ (y - p))
        # define B_{t+1} = B_{t} - H^{-1} S = B_{t} + (XWX^T)^{-1}X(y-p)
        beta_ = beta + inc
        norm_ = norm(beta_, beta)
        if norm_ < tol: # convergence check
            converged = True
            return beta_
        else:
            beta = beta_
            count = count + 1

beta = get_beta(X_train,y_train, max_iter = 1000)


We report the first 5 components of the optimum value of the logistic parameter $\beta$

In [7]:
beta[0:5]

array([[-0.07138813],
       [-0.00188321],
       [ 0.00781259],
       [ 0.01185498],
       [ 0.00177207]])

As we only have 2 classes, 1 or 0, $L_{hat}(h) = 1/n (\Sigma(I|h(x_i)-y_i|=1))$

In [8]:
def error(y_true, y_pred):
    return np.mean(np.abs(y_true - y_pred))

We now determine the resulting training error and test error

In [9]:
y_pred_train = (1 * (sigmoid(beta.T @ X_train.T) >= 0.5)).reshape(-1,1)
training_error = error(y_true = y_train, y_pred = y_pred_train)

print('Training error: {}%'.format(round(training_error * 100,4)))

Training error: 3.0259%


In [10]:
y_pred_test = (1 * (sigmoid(beta.T @ X_test.T) >= 0.5)).reshape(-1,1)
test_error = error(y_true = y_test, y_pred = y_pred_test)

print('Test error: {}%'.format(round(test_error * 100,4)))

Test error: 48.411%
